spin-chain consists of N $1/2-spins/qubits$ in a magnetic field
Each spin interacts with neighbours


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from qutip import (about, basis, expect, mesolve, qeye, sigmax, sigmay, sigmaz,
                   tensor)

%matplotlib inline